In [1]:
# Load packages
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/intermediate/sod'

# Set path to be the directory:
os.chdir(path)

In [3]:
# Read the list of county codes:
cw = pd.read_excel('qcew-county-msa-csa-crosswalk.xlsx', sheet_name='Feb. 2013 Crosswalk')

cw['STCNTYBR'] = cw['County Code'].astype(str)
cw.loc[cw['STCNTYBR'].str.len() == 4, 'STCNTYBR'] = '0' + cw['STCNTYBR']

In [4]:
# Load branch data:
sod = pd.read_csv('sod_data.csv', header=0, sep=',')
#inst = pd.read_csv('sod_data_ins.csv', header=0, sep=',')

C:\Users\angel\AppData\Local\Temp\ipykernel_20364\2809252724.py:2: DtypeWarning: Columns (13,28,50,59,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  sod = pd.read_csv('sod_data.csv', header=0, sep=',')


In [5]:
sod['STCNTYBR'] = sod['STCNTYBR'].astype(str)
sod.loc[sod['STCNTYBR'].str.len() == 4, 'STCNTYBR'] = '0' + sod['STCNTYBR']

In [6]:
# Merge 'list' on 'STCNTYBR' with the sod data:
sod = pd.merge(sod, cw, on='STCNTYBR', how='left')

In [7]:
# show columns 13,28,50,59,60,62:
cols = [13,28,50,59,60,62]

# Specify dtypes for col 13, 28, 50, 59, 60, 62:
sod.iloc[:,cols].dtypes

DEPSUMBR    object
NECNAMB     object
ASSET       object
DEPDOM      object
DEPSUM      object
ESCROW      object
dtype: object

In [8]:
# Take commas away from columns 13, 28, 50, 59, 60, 62 so we can convert them to numeric:
sod.iloc[:,cols] = sod.iloc[:,cols].replace(',', '', regex=True)
#inst.iloc[:,cols] = inst.iloc[:,cols].replace(',', '', regex=True)

In [9]:
# Convert DEPSUMBR to numeric:
sod['DEPSUMBR'] = pd.to_numeric(sod['DEPSUMBR'], errors='coerce')

In [10]:
sod.to_csv('sod_clean.csv', index=False)

In [ ]:
# Create an ID variable that is equal to RSSDHCR, and if RSSDHCR==0 then ID=RSSDID:
sod['ID'] = np.where(sod['RSSDHCR']==0, sod['RSSDID'], sod['RSSDHCR']) 

In [ ]:
sod.groupby('RSSDID')['RSSDID'].head()

In [ ]:
# Is there a 1 to 1 map between RSSDID and CERT numbers?
sod.groupby('RSSDID').CERT.nunique().value_counts()


# Matches with Jason

In [ ]:
sod[sod['RSSDID']==480228]['CERT'].unique()

In [ ]:
sod[sod['RSSDID']==480228]['NAMEFULL'].unique()

In [ ]:
sod[(sod['RSSDID']==480228) & (sod['NAMEFULL']=='Bank of America, National Association') & (sod['CERT']==3510)]['YEAR'].unique()

In [ ]:
sod[(sod['RSSDID']==480228) & (sod['NAMEFULL']=='NationsBank, National Association') & (sod['CERT']==15802)]['YEAR'].unique()

In [ ]:
sod[(sod['RSSDID']==480228) & (sod['NAMEFULL']=='NationsBank, National Association (Carolinas)') & (sod['CERT']==15802)]['YEAR'].unique()

In [ ]:
sod[(sod['RSSDID']==480228) & (sod['NAMEFULL']=='Nationsbank of North Carolina, National Association') & (sod['CERT']==4892)]['YEAR'].unique()

In [ ]:
# Value in USD 
print('Deposits in 1996:', sod[sod['YEAR']==1996]['DEPSUMBR'].sum())

# Matches with Jason 

In [ ]:
# Create dataset excluding the HQ and NaN for the MSA Title:
df = sod[(sod['BKMO']==0) & (sod['MSA Title'].notnull())]

In [ ]:
# Aggregate deposits over MSA per year, it will be useful later...
df_grouped_msa = df.groupby(['MSA Title', 'YEAR'])['DEPSUMBR'].sum().reset_index(name='total_msa')

# Aggregate all deposits over US per year:
df_grouped_msa['total_US'] = df_grouped_msa.groupby('YEAR')['total_msa'].transform('sum')

# Share of deposits in each MSA, per year:
df_grouped_msa['share_msa'] = df_grouped_msa['total_msa']/df_grouped_msa['total_US']

In [ ]:
# Share of deposits in each MSA, per year:
df_grouped_msa['share_msa'] = df_grouped_msa['total_msa']/df_grouped_msa['total_US']

In [ ]:
# Sanity check, sum of share_msa per year should be 1:
df_grouped_msa.groupby('YEAR')['share_msa'].sum()

In [ ]:
# Compute the share of deposits per ID and year, note that the total deposits per MSANAMB is in the df_grouped dataset:
df = pd.merge(df, df_grouped_msa, on=['MSA Title', 'YEAR'], how='left')

# Sort values by MSA and year:
df = df.sort_values(by=['MSA Title', 'YEAR'])

In [ ]:
df[df['ID']==0] # matches with jason until here

In [ ]:
# Write a .csv file with df[df['ID']==0]:
df[df['ID']==0].to_csv('sod_data_no_id.csv', index=False)

In [ ]:
df[df['ID']==0]['RSSDHCR']

In [ ]:
# Compute the total amount of deposits per NAMEHCR, in each MSA and year:
df_grouped_id = df.groupby(['ID', 'MSA Title', 'YEAR'])['DEPSUMBR'].sum().reset_index(name='total_id')

df = pd.merge(df, df_grouped_id, on=['ID', 'MSA Title', 'YEAR'], how='left')

In [ ]:
# Compute share of deposits per NAMEHCR, in each MSA and year:
df['share_id'] = 100*(df['total_id']/df['total_msa'])
df['share_id_sq'] = df['share_id']**2


In [ ]:
# Take the columns ID, YEAR, MSANAMB, and share_id_sq and take uniques on IDs, keep only those columns:
df_grouped_id_sq = df[['ID', 'YEAR', 'MSA Title','share_id_sq']].drop_duplicates()
 

In [ ]:
# Sum all the share_id_sq per MSANAMB and year:
df_grouped_id_sq = df_grouped_id_sq.groupby(['MSA Title', 'YEAR'])['share_id_sq'].sum().reset_index(name='HHI')

In [ ]:
df_grouped_id_sq[df_grouped_id_sq['HHI']==10000]

In [ ]:
df_grouped_id_sq[df_grouped_id_sq['HHI']==10000]

In [ ]:
# sum the squared shares of deposits per MSA, per year:
df_HHI = df_grouped_id_sq[['YEAR', 'MSA Title', 'HHI']].drop_duplicates()

In [ ]:
df_HHI = pd.merge(df_HHI, df_grouped_msa, on=['MSA Title', 'YEAR'], how='left')

In [ ]:
# plot a scatter plot of the HHI per share_msa per year, I want the dots for each year to have different collors, and 
# to be in different plots:
df_HHI_some_years = df_HHI[(df_HHI['YEAR']==1994) | (df_HHI['YEAR']==2010) | (df_HHI['YEAR']==2020) ]
sns.scatterplot(data=df_HHI_some_years, x='share_msa', y='HHI', hue='YEAR', palette='seismic')
plt.legend()
plt.title('HHI vs Share of deposits per year')
plt.xlabel('Share of deposits')
plt.ylabel('HHI')
plt.savefig('HHI_vs_share_deposits.png')
plt.show()


In [ ]:
# I want to make the same plot as above, but only for 1994 and 2022:
df_HHI_2000 = df_HHI[(df_HHI['YEAR']==2000)]

sns.scatterplot(data=df_HHI_2000, x='share_msa', y='HHI', hue='YEAR', palette='seismic')
plt.legend()
plt.title('HHI vs Share of deposits per year')
plt.xlabel('Share of deposits')
plt.ylabel('HHI')


In [ ]:
# I want to make the same plot as above, but only for 1994 and 2022:
df_HHI_2010 = df_HHI[(df_HHI['YEAR']==2010)]

sns.scatterplot(data=df_HHI_2010, x='share_msa', y='HHI', hue='YEAR', palette='seismic')
plt.legend()
plt.title('HHI vs Share of deposits per year')
plt.xlabel('Share of deposits')
plt.ylabel('HHI')


In [ ]:
# I want to make the same plot as above, but only for 1994 and 2022:
df_HHI_2020 = df_HHI[(df_HHI['YEAR']==2020)]

sns.scatterplot(data=df_HHI_2020, x='share_msa', y='HHI', hue='YEAR', palette='seismic')
plt.legend()
plt.title('HHI vs Share of deposits per year')
plt.xlabel('Share of deposits')
plt.ylabel('HHI')


In [ ]:
# Aggregate to get one HHI per year, averaging over MSAs:
df_HHI = df_HHI.groupby('YEAR')['HHI'].mean().reset_index(name='HHI')

# Plot HHI over time:
plt.plot(df_HHI['YEAR'], df_HHI['HHI'], color='black')
plt.xlabel('Year')
plt.ylabel('HHI')
plt.title('HHI over time')
plt.grid(True, color='lightgray', linestyle='-', linewidth=0.5, alpha=0.5)
plt.savefig('HHI_over_time.svg')
plt.show()

In [ ]:
# Plot the histogram of HHI for 1994, 2010, and 2020. Please use fraction of observations in the y-axis:
plt.hist(df_HHI_2000['HHI'], bins=100, alpha=1, label='2000', density=True)
plt.hist(df_HHI_2010['HHI'], bins=100, alpha=1, label='2010', density=True)
plt.hist(df_HHI_2020['HHI'], bins=100, alpha=1, label='2020', density=True)
plt.legend()
plt.xlabel('HHI')
plt.ylabel('Density')
plt.title('HHI distribution for 1994, 2010, and 2020')
plt.savefig('HHI_distribution.png')
plt.show()




 

In [ ]:
# temporary dataset
temp = df[['ID', 'YEAR', 'MSA Title', 'share_id_sq', 'share_msa']].drop_duplicates()

# compute HHI per MSA and year
wHHI = temp.groupby(['MSA Title', 'YEAR'])['share_id_sq'].sum().reset_index(name='HHI')

df_wHHI = pd.merge(df[['MSA Title', 'YEAR', 'share_msa']], wHHI, on=['MSA Title', 'YEAR'], how='left').drop_duplicates()


In [ ]:
# Average HHI over MSAs, weighting by 'share_msa' to get one value per year:
df_wHHI['wHHI'] = df_wHHI['HHI']*df_wHHI['share_msa']

# sum wHHI variable per year:
df_wHHI = df_wHHI.groupby('YEAR')['wHHI'].sum().reset_index(name='wHHI')


In [ ]:
# plot wHHI over time:
plt.plot(df_wHHI['YEAR'], df_wHHI['wHHI'], color='black')
plt.xlabel('Year')
plt.ylabel('Weighted HHI')
plt.title('HHI weighted by deposits over time')
plt.grid(True, color='lightgray', linestyle='-', linewidth=0.5, alpha=0.5)
plt.savefig('wHHI_over_time.svg')
plt.show()